# Extract data from FIRMS for near-realtime wild fires in Canada

In [1]:
from datetime import datetime, timedelta
start_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')  # Today's date
start_date

'2025-07-13'

In [2]:
import folium
import ee
from datetime import datetime, timedelta

# Initialize the Earth Engine library
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='eo4c-day01')

# Function to add Earth Engine layers to Folium
def add_ee_layer(self, ee_object, vis_params, name):
    map_id_dict = ee.Image(ee_object).getMapId(vis_params)
    tile_url = map_id_dict['tile_fetcher'].url_format
    folium.TileLayer(
        tiles=tile_url,
        attr="Map Data: Google Earth Engine",
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the function to the Folium Map class
folium.Map.add_ee_layer = add_ee_layer

# Define the region of interest (Canada)
# canada_geometry = ee.Geometry.Polygon(
#         [[[-141.0, 41.0], [-141.0, 83.0], [-52.0, 83.0], [-52.0, 41.0]]])
canada_geometry = ee.Geometry.Polygon(
    [[[-77.807, 55.529], [-95.229, 55.529], [-95.229, 48.814], [-77.807, 48.814]]]
)

# Define the date range
# start_date = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d')  # Today's date
start_date = '2025-05-01'
end_date = datetime.today().strftime('%Y-%m-%d')  # Today's date

# Load the FIRMS dataset (MODIS/VIIRS fires)
firms = ee.ImageCollection("FIRMS")

# Filter the data by date range and region (Canada)
fires = firms.filterBounds(canada_geometry) \
             .filterDate(start_date, end_date)

# Get the median composite image (fire data)
fire_image = fires.median()

# Select the specific band for visualization (e.g., 'fire' band)
# If the composite image has multiple bands, we will select the 'fire' band or the first band
# You can change the band name to something else if it differs in your dataset
fire_band = fire_image.select(['T21'])

# Visualization parameters (adjust as needed)
vis_params = {
    'min': 0,
    'max': 2,
    'palette': ['blue', 'green', 'yellow', 'red']
}

# Create a Folium map centered on Canada
map_center = [56.1304, -106.3468]  # Coordinates for Canada
map_folium = folium.Map(location=map_center, zoom_start=5)

# Add the Earth Engine fire layer to the map
map_folium.add_ee_layer(fire_band, vis_params, "Canada Wildfires")

# Add a layer control to toggle layers
map_folium.add_child(folium.LayerControl())

# Display the map
map_folium

